In [1]:
import pandas as pd
import hydrant.ZonalProcessing.ZonalProcessing as zp

In [2]:
# Sample DataFrame
data = {
    'COMID': [28070338, 28070572, 28072028, 28072168, 28072213],
    'frac_000': [0.000395, 0.0, 0.0, 0.0, 0.0],
    'frac_003': [0.692559, 0.184927, 0.804716, 0.939592, 0.0],
    'frac_002': [0.277206, 0.815073, 0.195284, 0.060349, 0.0],
    'frac_008': [0.004143, 0.0, 0.0, 0.0, 0.0],
    'frac_011': [0.016663, 0.0, 0.0, 0.000059, 0.0],
    'frac_012': [0.009034, 0.0, 0.0, 0.0, 0.0],
    'others': ['aa', 'aa', 'aa', 'aa', 'aa']
}

df = pd.DataFrame(data)

info = {'ID': 'COMID',
        'name_of_dataset': 'soil_class',
        'explanation_of_dataset': '',
        'prefix': 'frac'}

# Mapping dictionary
mapping = {
    1: 'clay',
    2: 'silty clay',
    3: 'sandy clay',
    4: 'clay loam',
    5: 'silty clay loam',
    6: 'sandy clay loam',
    7: 'loam',
    8: 'silty loam',
    9: 'sandy loam',
    10: 'silt',
    11: 'loamy sand',
    12: 'sand'
}

ds_modified, df_modified = zp.Fraction_to_Xarray(df, info, mapping)

ds_modified

<xarray.Dataset>
Dimensions:              (COMID: 5, soil_class: 5)
Coordinates:
  * COMID                (COMID) int64 28070338 28070572 ... 28072168 28072213
  * soil_class           (soil_class) int64 2 3 8 11 12
Data variables:
    soil_class_majority  (COMID) int64 3 2 3 3 3
    soil_class_frac      (COMID, soil_class) float64 0.2773 0.6928 ... 0.0 0.0
    soil_class_names     (soil_class) <U10 'silty clay' 'sandy clay' ... 'sand'

In [3]:
# Sample DataFrame
data = {
    'COMID': [28070338, 28070572, 28072028, 28072168, 28072213],
    'mean':  [510, 560, 580, 600, 1000],
    'max':   [600, 650, 680, 700, 1100],
    'min':   [400, 450, 480, 500,  900],
}

df = pd.DataFrame(data)

info = {'ID': 'COMID'}

# Mapping dictionary
mapping = {'mean':{'units':'m'},
           'max':{'units':'m'},
           'min':{'units':'m'}}

ds = zp.Stat_to_Xarray(df, info, mapping)
ds

<xarray.Dataset>
Dimensions:  (COMID: 5)
Coordinates:
  * COMID    (COMID) int64 28070338 28070572 28072028 28072168 28072213
Data variables:
    mean     (COMID) int64 510 560 580 600 1000
    max      (COMID) int64 600 650 680 700 1100
    min      (COMID) int64 400 450 480 500 900

In [5]:
# Sample DataFrames
data1 = {
    'ID1': [1,   2,  3],
    'A_1': [0.3, 0,  0],
    'A_2': [0,   0,  0],
    'A_3': [0.5, 0,  0],
    'A_4': [0,   1,  0],
    'A_5': [0.2, 0,  1],
    'B':   ['B','B','B'],
}

data2 = {
    'ID2': [1, 2, 3  ],
    'B1':  [0, 1, 0.5],
    'B2':  [1, 0, 0.5]
}

data3 = {
    'ID3': [1, 2,   3],
    'C1':  [0, 1, 0.3],
    'C2':  [1, 0, 0.7],
    'D':   [1, 1,   1]
}

# Create DataFrames
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
df3 = pd.DataFrame(data3)

# Usage example:
result, report, ds= zp.intersect_df(df1, df2, df3, 
                                    df_mappings={'df1': {'id': 'ID1', 'prefix':'A' , 'data_name':'LULC'}, 
                                                 'df2': {'id': 'ID2', 'prefix':'B' , 'data_name':'S'}, 
                                                 'df3': {'id': 'ID3', 'prefix':'C' , 'data_name':'ASP'}},
                                    remove_zero_combinations = True)

print(result)
print(report)
ds

The indexes of all DataFrames are exactly the same with the same order.
total number of non zero combinations:  7
     comb_0001  comb_0002  comb_0003  comb_0004  comb_0005  comb_0006  \
ID1                                                                     
1          0.3        0.5        0.0       0.00       0.00       0.00   
2          0.0        0.0        1.0       0.00       0.00       0.00   
3          0.0        0.0        0.0       0.15       0.35       0.15   

     comb_0007  
ID1             
1         0.20  
2         0.00  
3         0.35  
     Combinations LULC  S ASP  comb
0  LULC_1 S2 ASP2    1  2   2     1
1  LULC_3 S2 ASP2    3  2   2     2
2  LULC_4 S1 ASP1    4  1   1     3
3  LULC_5 S1 ASP1    5  1   1     4
4  LULC_5 S1 ASP2    5  1   2     5
5  LULC_5 S2 ASP1    5  2   1     6
6  LULC_5 S2 ASP2    5  2   2     7


<xarray.Dataset>
Dimensions:       (m: 7, n: 3)
Coordinates:
  * m             (m) object 'comb_0001' 'comb_0002' ... 'comb_0006' 'comb_0007'
  * n             (n) int64 1 2 3
Data variables:
    Combinations  (m) object 'LULC_1 S2 ASP2' ... 'LULC_5 S2 ASP2'
    LULC          (m) object '1' '3' '4' '5' '5' '5' '5'
    S             (m) object '2' '2' '1' '1' '1' '2' '2'
    ASP           (m) object '2' '2' '1' '1' '2' '1' '2'
    comb          (m) object 'comb_0001' 'comb_0002' ... 'comb_0006' 'comb_0007'
    fraction      (n, m) float64 0.3 0.5 0.0 0.0 0.0 ... 0.0 0.15 0.35 0.15 0.35
    id            (n) int64 1 2 3